# Analizy Przestrzenne 2022Z
# Projekt 1: Wskazanie optymalnej lokalizacji inwestycji hotelowej – analizy wielokryterialne
dane wejściowe: 
* BDOT caly
* NMT sklejony dla gminy
* gazociag
* dane glebowo-rolnicze (gleby)
* granice gminy

dane wyjściowe:
* TBA

# Import bibliotek, ustawienie środowiska, ustawienie ścieżek do warstw

In [5]:
import sys
import arcpy
import os

In [6]:
# sprawdzanie rozszerzeń
# https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/checkextension.htm
arcpy.CheckOutExtension("Spatial")
arcpy.CheckOutExtension("3D")

'CheckedOut'

In [7]:
# nadpisywanie plików - nie działa, bo ESRI...
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

In [8]:
nazwa_gminy = "Świeradów-Zdrój"

# workspace_path = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\MyProject1.gdb"  # gdb
# folder_bdot = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\bdot_caly"
# path_nmt = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\nmt_gminy\nmt_swieradow.tif"
# path_gazociag = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\gazociag\gazociag.shp"
# path_gleby = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\gleby\gleby_gm_swieradow.shp"
# path_granice_gminy = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\granice_gminy\gmina_swieradow.gpkg\main.gmina_swieradow"
# path_dzialki = r"E:\DANE\AF\311512\arcgis_testy\MyProject1\DANE\dzialki_gminy_egib\dzialki_egib.gpkg"

workspace_path = r"C:\SEM5\AP\p1\analiza_hotele\analiza_hotele.gdb"  # gdb
folder_bdot = r"C:\SEM5\AP\p1\DANE\bdot_caly"
path_nmt = r"C:\SEM5\AP\p1\DANE\nmt_gminy\nmt_swieradow.tif"
path_gazociag = r"C:\SEM5\AP\p1\DANE\gazociag\gazociag.shp"
path_gleby = r"C:\SEM5\AP\p1\DANE\gleby\gleby_gm_swieradow.shp"
path_granice_gminy = r"C:\SEM5\AP\p1\DANE\granice_gminy\gmina_swieradow.gpkg\main.gmina_swieradow"
path_dzialki = r"C:\SEM5\AP\p1\DANE\dzialki_gminy_egib\dzialki_egib.gpkg"

bufor_gminy = arcpy.analysis.Buffer(path_granice_gminy, "bufor_gminy", 100)


In [9]:
# ustawienia środowiska pracy
# data paths
arcpy.env.workspace = workspace_path  # geodatabase
# processing extent
arcpy.env.extent = str(bufor_gminy)  # path to bufor_gminy
# output coordinate system
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("ETRS 1989 Poland CS92")
# cell size
arcpy.env.cellSize = 1
# nadpisywanie plików
arcpy.env.overwriteOutput = True

# Ogarnięcie nazw plików z folderu BDOT i zapisanie potrzebnych ścieżek

In [10]:
# Pozbycie się kropek z nazw plików w folderze bdot_caly

bdot = dict()
names = {'budynki':'bubd_a', 'drogi':'skdr_l', 'rzeki':'swrs_l', 'jeziora':'ptwp_a', 'lasy':'ptlz_a'}
extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
for filename in os.listdir(folder_bdot):
    if os.path.isfile(os.path.join(folder_bdot, filename)):
        src = rf"{folder_bdot}\{filename}"
        if src.endswith(extensions):
            if src.count('.') == 4:
                for k, v in names.items():
                    if v in filename.lower():
                        dst = src.replace('.', '', 3)
                        os.rename(src, dst)
                        if src.endswith('.shp'):
                            bdot[k] = dst

In [11]:
# Kreacja słownika bdot jesli juz mamy dobre nazwy warstw

bdot = dict()
names = {'budynki':'bubd_a', 'drogi':'skdr_l', 'rzeki':'swrs_l', 'jeziora':'ptwp_a', 'lasy':'ptlz_a'}
extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
for filename in os.listdir(folder_bdot):
    if os.path.isfile(os.path.join(folder_bdot, filename)):
        src = rf"{folder_bdot}\{filename}"
        if src.endswith(extensions):
            if src.count('.') == 1:
                for key, name in names.items():
                    if name in filename.lower() and src.endswith('.shp'):
                        bdot[key] = src

In [12]:
# bdot = {'budynki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_BUBD_A.shp', 'lasy': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTLZ_A.shp', 'jeziora': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTWP_A.shp', 'drogi': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SKDR_L.shp', 'rzeki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SWRS_L.shp'}
bdot

{'budynki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_BUBD_A.shp', 'lasy': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTLZ_A.shp', 'jeziora': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_PTWP_A.shp', 'drogi': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SKDR_L.shp', 'rzeki': 'C:\\SEM5\\AP\\p1\\DANE\\bdot_caly\\PLPZGiK3370210__OT_SWRS_L.shp'}

# Kryterium 1 - odległość od konkurencji minimum 400m

In [13]:
query = "FUNSZCZ LIKE '%hotel%' OR FUNSZCZ LIKE '%motel%' OR FUNSZCZ LIKE '%pensjonat%' OR FUNSZCZ LIKE '%domWypoczynkowy%'"
hotele = arcpy.analysis.Select(bdot['budynki'], "hotele", query)
ed = arcpy.sa.EucDistance(hotele)
minimum = 400 # 400
maximum = 800 # 800

k1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

# Kryterium 2 - odległość od budynków mieszkalnych od 25 do 150m

In [14]:
query = "FUNOGBUD LIKE '%miesz%' OR FUNOGBUD LIKE '%Miesz%'"  # 1036
mieszkalne = arcpy.analysis.Select(bdot['budynki'], "mieszkalne", query)
ed = arcpy.sa.EucDistance(mieszkalne)
minimum = 25 # 25
maximum = 50 # 50
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 150 # 150
maximum = 125 # 125
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k2 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 3 - odległość od istniejących dróg od 15 do 100m

In [15]:
drogi = bdot['drogi']
ed = arcpy.sa.EucDistance(drogi)
minimum = 15 # 15
maximum = 20 # 20
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 200 # 200
maximum = 100 # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k3 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 4 - odległość od rzek i zbiorników wodnych nieprzekraczalna strefa ochronna poniżej 20m

In [16]:
query = "X_KOD='SWRS01' OR X_KOD='SWRS02'"
rzeki = arcpy.analysis.Select(bdot['rzeki'], 'rzeki', query)
bufor_rzeki = arcpy.analysis.Buffer(rzeki, 'bufor_rzeki', 0.5)
jeziora = bdot['jeziora']
wody = arcpy.management.Merge([bufor_rzeki, jeziora], 'wody')
ed = arcpy.sa.EucDistance(wody)
minimum = 20 # 20
maximum = 20.001 # 25 20.001
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 1000 # 1000
maximum = 100 # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k4 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 5 - pokrycie terenu nie w lesie

In [17]:
lasy_raster = arcpy.conversion.FeatureToRaster(bdot['lasy'], "RODZAJ", 'lasy_raster')

k5 = arcpy.sa.Reclassify(lasy_raster, "RODZAJ", arcpy.sa.RemapValue([["las", 0], ["zadrzewienie", 1],["zagajnik", 0], ["NODATA", 1]]))

# Kryterium 6 - nachylenie stoków maksymalnie 20%

In [18]:
slope_perc = arcpy.sa.Slope(path_nmt, "PERCENT_RISE", 1, "PLANAR", "METER")
minimum = 20 # 20
maximum = 5 # 5

k6 = arcpy.sa.FuzzyMembership(slope_perc, arcpy.sa.FuzzyLinear(minimum, maximum))

# Kryterium 7 - dostęp światła słonecznego stoki południowe (SW-SE)

In [19]:
aspect = arcpy.sa.Aspect(path_nmt)
minimum = 110 # 110
maximum = 155 # 155
fuzzy1 = arcpy.sa.FuzzyMembership(aspect, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 250 # 250 
maximum = 205 # 205
fuzzy2 = arcpy.sa.FuzzyMembership(aspect, arcpy.sa.FuzzyLinear(minimum, maximum))

k7 = arcpy.sa.Times(fuzzy1, fuzzy2)
k7 = arcpy.sa.Con(k7, 1, k7, "VALUE = -1")

# Kryterium 8 - poza strefą ochronną gazociągu minimum 25m

In [20]:
ed = arcpy.sa.EucDistance(path_gazociag)
minimum = 25 # 25
maximum = 25.001 # 26 25.001
fuzzy1 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))
minimum = 1000 # 1000
maximum = 100  # 100
fuzzy2 = arcpy.sa.FuzzyMembership(ed, arcpy.sa.FuzzyLinear(minimum, maximum))

k8 = arcpy.sa.Times(fuzzy1, fuzzy2)

# Kryterium 9 - użytkowanie terenu - przydatne gleby

In [33]:
codeblock = """
def is_useful(string):
    if string in ['G','E','M','T','F','Fb','Fc','FG']:
        return "NIE"  # 0
    return "TAK"  # 1
"""
expression = "is_useful(!TYP!)"
field_name = "przydatne"
gleby = arcpy.management.CalculateField(path_gleby, field_name, expression,"PYTHON3", codeblock)

k9 = arcpy.conversion.FeatureToRaster(gleby, "przydatne", "k9")
k9 = arcpy.sa.Reclassify(k9, "przydatne", arcpy.sa.RemapValue([["NIE", 0], ["TAK", 1]]))

# Łączenie kryteriów

### Zrobienie kryteriów ostrych ze stref ochronnych wodociągu i wód

In [34]:
# 2 + 2 = 4 (5) ostrych (lasy, gleby i dodatkowo strefy ochronne wód i gazociągu)
# Weighted sum do fuzzy 7/8 k1 k2 k3 k4 k6 k7 k8
# Fuzzy overlay do ostrych 4/5 k5 k9 k4_ostre k8_ostre
# i potem mnozymy funkcją Times
# lub Raster Calculator
# dwa scenariusze - jednakowe wagi i wagi eksperckie

In [35]:
ed = arcpy.sa.EucDistance(wody)
k4_ostre = arcpy.sa.Con(ed, 1, 0, "VALUE > 20")

In [36]:
ed = arcpy.sa.EucDistance(path_gazociag)
k8_ostre = arcpy.sa.Con(ed, 1, 0, "VALUE > 25")

### Scenariusz 1: wagi jednakowe

In [49]:
wstable_1 = arcpy.sa.WSTable([[k1,"VALUE",0.1428],[k2,"VALUE",0.1428],[k3,"VALUE",0.1428],[k4,"VALUE",0.1428],[k6,"VALUE",0.1428],[k7,"VALUE",0.1428],[k8,"VALUE",0.1432]])
joined_fuzzy_1 = arcpy.sa.WeightedSum(wstable_1)

In [50]:
fo_ostre = arcpy.sa.FuzzyOverlay([k5, k9, k4_ostre, k8_ostre], "AND")  # "AND"

In [51]:
final_fuzzy_1 = arcpy.sa.Times(joined_fuzzy_1, fo_ostre)

In [57]:
ff1_reclass = arcpy.sa.Con(final_fuzzy_1, 1, 0, "VALUE > 0.5")  # VALUE > 0.7

### Scenariusz 2: wagi eksperckie

* Kryterium 1 - ```0.2``` - odległość od konkurencji minimum 400m
* Kryterium 2 - ```0.05``` - odległość od budynków mieszkalnych od 25 do 150m
* Kryterium 3 - ```0.05``` - odległość od istniejących dróg od 15 do 100m
* Kryterium 4 - ```0.2``` - odległość od rzek i zbiorników wodnych
* Kryterium 6 - ```0.15``` - nachylenie stoków maksymalnie 20%
* Kryterium 7 - ```0.3``` - dostęp światła słonecznego stoki południowe (SW-SE)
* Kryterium 8 - ```0.05``` - odległość od gazociągu

In [53]:
wstable_2 = arcpy.sa.WSTable([[k1,"VALUE",0.2],[k2,"VALUE",0.05],[k3,"VALUE",0.05],[k4,"VALUE",0.2],[k6,"VALUE",0.15],[k7,"VALUE",0.3],[k8,"VALUE",0.05]])
joined_fuzzy_2 = arcpy.sa.WeightedSum(wstable_2)

In [54]:
fo_ostre = arcpy.sa.FuzzyOverlay([k5, k9, k4_ostre, k8_ostre], "AND")  # "AND"

In [55]:
final_fuzzy_2 = arcpy.sa.Times(joined_fuzzy_2, fo_ostre)

In [58]:
ff2_reclass = arcpy.sa.Con(final_fuzzy_2, 1, 0, "VALUE > 0.5")  # VALUE > 0.7

# Kryterium 10 - przydatne działki obszary 1-6ha

# Kryterium 11 - kształt obszaru jak najbardziej zwarty

# Cośtam z mapą kosztów

In [60]:
costs_dict = {
"PTWP01": 0,
"PTWP02": 200,
"PTWP03": "0 NoData",
"PTZB01": 200,
"PTZB02": 100,
"PTZB03": 200,
"PTZB04": 200,
"PTZB05": 50,
"PTLZ01": 100,
"PTLZ02": 50,
"PTLZ03": 50,
"PTRK": 15,
"PTRK02": 15,
"PTUT01": "0 NoData",
"PTUT02": 90,
"PTUT03": 100,
"PTUT04": 20,
"PTUT05": 20,
"PTTR01": 20,
"PTTR02": 1,
"PTKM01": 100,
"PTKM02": 200,
"PTKM03": 170,
"PTKM04": 200,
"PTGN01": 1,
"PTGN02": 1,
"PTGN03": 1,
"PTGN04": 1,
"PTPL01": 50,
"PTSO01": "0 NoData",
"PTSO02": "0 NoData",
"PTWZ01": "0 NoData",
"PTWZ02": "0 NoData",
"PTNZ01": 150,
"PTNZ02": 150}

# TRASH

In [26]:
# ?arcpy.analysis.Select

In [27]:
# arcpy.Describe(bufor_gminy)

In [28]:
# # TYLKO RAZ - pozbycie się kropek z nazw plików w folderze bdot_caly
# bdot = dict()
# extensions = ('.shp', '.prj', '.xlsx', '.dbf', '.shx', '.cpg')
# for filename in os.listdir(folder_bdot):
#     if os.path.isfile(os.path.join(folder_bdot, filename)):
#         src = rf"{folder_bdot}\{filename}"
#         if src.endswith(extensions):
#             if src.count('.') == 4:
#                 if "bubd_a" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['budynki'] = dst
#                 if "skdr_l" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['drogi'] = dst
#                 if "swrs_l" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['rzeki'] = dst
#                 if "ptwp_a" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['jeziora'] = dst
#                 if "ptlz_a" in filename.lower():
#                     dst = src.replace('.', '', 3)
#                     os.rename(src, dst)
#                     if src.endswith('.shp'):
#                         bdot['lasy'] = dst